Problem statement:

Situation: Energy is produced in different forms and consumed by the regions in different rates.   

Complication: Energy prices are increasing due to many reasons including but not limited to war, logistics and sanctions.   

Question: Is it possible to see what types of energy have been historically produced and consumed by the different regions? Is it possible to predict how the productions and consumption will look like in the future?   

Answer: ???


In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
sns.set_theme(style='darkgrid', palette='crest')
import matplotlib.pyplot as plt



Installing CmdStan version: 2.31.0
Install directory: C:\Users\markella.nikolopoulo\.cmdstan
Download successful, file: C:\Users\MARKEL~1.NIK\AppData\Local\Temp\tmpe3nc26ix
Extracting distribution
Unpacked download as cmdstan-2.31.0
Building version cmdstan-2.31.0, may take several minutes, depending on your system.
Test model compilation
Installed cmdstan-2.31.0


18:14:58 - cmdstanpy - WARNING - CmdStan installation failed.
cannot import name 'run_rtools_install' from 'cmdstanpy.install_cxx_toolchain' (c:\ProgramData\Anaconda3\envs\thisone\lib\site-packages\cmdstanpy\install_cxx_toolchain.py)


Installing CmdStan version: 2.31.0
Install directory: C:\Users\markella.nikolopoulo\.cmdstan


False

In [18]:
leveranser_flytande = pd.read_table('../data/to-use/leveranser-flytande-bransle-region-ar.csv', sep=',')
elproduction_bransleanvandning = pd.read_table('../data/to-use/elproduction-bransleanvandning-region-produktionssatt-bransletyp-ar.csv', sep=',')
fjarrvarmeproduktion = pd.read_table('../data/to-use/fjarrvarmeproduktion-region-produktionssatt-bransletyp-ar.csv', sep=',')
slutanvandning = pd.read_table('../data/to-use/slutanvandning-region-forbrukarkategori-bransletyp-kategori-energityp-ar.csv', sep=',')
bransleforbrukning = pd.read_table('../data/to-use/branslefarbrukning-far-produktion-per-ar.csv', sep=',')
energidata = pd.read_table('../data/to-use/energidata-per-ar.csv', sep=',')

datasets = [leveranser_flytande, elproduction_bransleanvandning, 
            fjarrvarmeproduktion, slutanvandning, bransleforbrukning, energidata]

variables = ['region', 'bransletyp', 'ar', 'm3', 'farbrukarkategori', 'produktionssatt',
             'MWh', 'kategori', 'energityp', 'produktionsslag', 'bransleforbukning']

## FIll NAs
leveranser_flytande.m3 = leveranser_flytande.m3.fillna(method='ffill')
fjarrvarmeproduktion.MWh = fjarrvarmeproduktion.MWh.interpolate()
# slutanvandning = slutanvandning.fillna('fffill')
energidata.MWh = energidata.MWh.fillna(method='ffill')


# Slice dataset for regions
leveranser_flytande = leveranser_flytande[leveranser_flytande.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
elproduction_bransleanvandning = elproduction_bransleanvandning[elproduction_bransleanvandning.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
fjarrvarmeproduktion = fjarrvarmeproduktion[fjarrvarmeproduktion.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
slutanvandning = slutanvandning[slutanvandning.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
energidata = energidata[energidata.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)



## Group datasets
leveranser_flytande = leveranser_flytande.groupby(['region', 'ar', 'fornybar_bransletyp'], as_index=False).agg(Sum=('m3', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
elproduction_bransleanvandning_grouped = elproduction_bransleanvandning.groupby(['region', 'ar', 'fornybar_bransletyp', 'fornybar_produktionssatt'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
fjarrvarmeproduktion = fjarrvarmeproduktion.groupby(['region', 'ar', 'fornybar_bransletyp', 'fornybar_produktionssatt'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
slutanvandning = slutanvandning.groupby(['region', 'ar', 'fornybar_bransletyp', 'forbrukarkategori'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
bransleforbrukning = bransleforbrukning.groupby(['ar', 'fornybar_bransletyp', 'produktionsslag'], as_index=False).agg( Sum=('branslefarbrukning', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
energidata = energidata.groupby(['region', 'ar', 'fornybar_energityp', 'fornybar_kategori'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)


leveranser_flytande.ar = pd.to_datetime(leveranser_flytande.ar, format='%Y-%m-%d').dt.year
elproduction_bransleanvandning_grouped.ar = pd.to_datetime(elproduction_bransleanvandning_grouped.ar, format='%Y-%m-%d').dt.year
fjarrvarmeproduktion.ar = pd.to_datetime(fjarrvarmeproduktion.ar, format='%Y-%m-%d').dt.year
slutanvandning.ar = pd.to_datetime(slutanvandning.ar, format='%Y-%m-%d').dt.year
bransleforbrukning.ar = pd.to_datetime(bransleforbrukning.ar, format='%Y-%m-%d').dt.year
energidata.ar = pd.to_datetime(energidata.ar, format='%Y-%m-%d').dt.year



C:\Users\markella.nikolopoulo\AppData\Local\Temp\ipykernel_18592\3935031558.py:4: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  slutanvandning = pd.read_table('../data/to-use/slutanvandning-region-forbrukarkategori-bransletyp-kategori-energityp-ar.csv', sep=',')


In [ ]:
# ## Prophet

# from prophet import Prophet
# from prophet.plot import plot_plotly, plot_components_plotly
# import cmdstanpy
# cmdstanpy.install_cmdstan()
# cmdstanpy.install_cmdstan(compiler=True)

# region_datasets = [leveranser_flytande, elproduction_bransleanvandning, 
#             fjarrvarmeproduktion, slutanvandning, energidata]
# # # for d in region_datasets:
# # #     name = [x for x in globals() if globals()[x] is d][0]
# # #     print('Dataframe: ', name)
    
# # for region in energidata.region.drop_duplicates(keep='first').reset_index(drop=True):
# #     print(region)
# #     data = energidata[energidata.region == region].reset_index(drop=True)
# #     for fornybar_energityp in energidata.fornybar_energityp.drop_duplicates(keep='first').reset_index(drop=True):
# #         print(fornybar_energityp)
# #         data = data[data.fornybar_energityp == fornybar_energityp].reset_index(drop=True)
# #         for fornybar_kategori in energidata.fornybar_kategori.drop_duplicates(keep='first').reset_index(drop=True):
# #             print(fornybar_kategori)
# #             data = data[data.fornybar_kategori == fornybar_kategori].reset_index(drop=True)

# #             data['ds'] = data.ar
# #             data['y'] = data.Sum
# #             print(data)
# #             m = Prophet()
# #             m.fit(data)

# #             future = m.make_future_dataframe(periods=365)
# #             future.tail()
# #             forecast = m.predict(future)
# #             forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# #             fig1 = m.plot(forecast)

# #             fig2 = m.plot_components(forecast)


# #             plot_plotly(m, forecast)
# #             plot_components_plotly(m, forecast)
